In [1]:
import os
os.chdir('../')

In [2]:
from config import sky
from skydb.sheets import *
from google_sheets import *
import pandas as pd

In [5]:
sky.get()

,id,base_role_id,hidden,name
0,67454,1,False,Academic Chair
1,58802,523,False,Academic Group Manager
2,66719,10211,False,Academic Holds Manager
3,66648,2,False,Academic Platform
4,58820,3177,False,Activity Group Manager
...,...,...,...,...
108,55569,1455,False,System Group Manager
109,55525,15,False,Teacher
110,55536,556,False,Trustee
111,61972,2,False,User Managment


In [6]:
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.bucket('sky-api-326214.appspot.com')
BLOB = bucket.blob('.sky-token')
# Saving the token to Google Cloud Storage
BLOB.upload_from_filename('/tmp/.sky-token')

In [ ]:
def getOfficialRecords():
    TRANSCRIPT_GRADES = 74465
    TRANSCRIPT_TRANSFER_GRADES = 74464

    transcript_grades = sky.getAdvancedList(TRANSCRIPT_GRADES)
    transcript_transfer_grades = sky.getAdvancedList(TRANSCRIPT_TRANSFER_GRADES)

    """ Adding GPA Points from offical transcript grades """
    gpa_points = (transcript_grades[['gpa_points', 'grade']]
    .drop_duplicates()
    .dropna()
    )

    transcript_transfer_grades = (
        transcript_transfer_grades
        .merge(
            gpa_points,
            'inner',
            on='grade'
        )
    )

    """ Combining row data from the two dfs """
    student_records = (
        pd.concat(
        [transcript_grades[list(transcript_grades.columns)], 
        transcript_transfer_grades[list(transcript_transfer_grades.columns)]],
        axis=0
        )
        .sort_values(['last_name', 'first_name', 'user_id', 'school_year', 'term'])
        .query("level_description == 'Middle School'")
        .reset_index(drop=True)
        .astype({
            'user_id':'int64',
            'credits_attempted':'float',
            'credits_earned':'float',
            'offering_id':'int64',
            'gpa_points':'float',
            'weight':'float'
        })
    )

    """ Filtering out any US Students """
    students = sky.getUsers()

    middleschoolers = (
        students[['id', 'student_info.grade_level_abbreviation']]
        .astype('int64')
        .rename(columns={
        'id':'user_id',
        'student_info.grade_level_abbreviation':'grade_level'
        })
        .query("grade_level < 9 and grade_level > 6")
        .drop('grade_level', axis=1)
        .user_id
        .tolist()
    )

    student_records = student_records[student_records.user_id.isin(middleschoolers)]

    """ Cleaning Credits Earned/Attempted"""
    for index, row in student_records.iterrows():
        if not row['grade'] or row.course_title in [
            "Grade 6 Art", "Grade 6 Band Chorus", "Grade 6 Robotics",
            'Band Chorus (6)', 'Robotics (6)', 'Art (6)'
        ]:
            # The grade is incomplete so no credits are taken or earned
            student_records.loc[index, ['credits_attempted', 'credits_earned']] = 0
            continue
        # Student completed the course and attempted 0.5 credits
        student_records.loc[index, 'credits_attempted'] = 0.5
        if row['grade'] not in ['Fail', 'F']:
            # Student passed and earned 0.5 credits
            student_records.loc[index, 'credits_earned'] = 0.5 
        else:
            # Student Failed and earned a 0
            student_records.loc[index, 'credits_earned'] = 0

    """ Getting Students GPAs """
    student_gpas = (
        student_records
        .assign(unweighted_numerator=lambda x: x['gpa_points'] * x['credits_earned'],
            weighted_numerator=lambda x: (x['gpa_points'] + x['weight']) * x['credits_earned']
            )
        .groupby('user_id')
        .sum()
        .assign(unweighted_gpa=lambda x: round(x['unweighted_numerator'] / x['credits_earned'], 2),
            weighted_gpa=lambda x: round(x['weighted_numerator'] / x['credits_earned'], 2)
            )
        [['unweighted_gpa', 'weighted_gpa']]
        .reset_index()
    )

    return {
        'gpas': student_gpas,
        'grades':student_records.drop(['first_name', 'last_name', 'level_description'], axis=1)
    }


In [ ]:
records = getOfficialRecords()

In [ ]:
qualified_students = []
for index, row in records['gpas'].iterrows():
    duration = len(records['grades']
     .query(f"""user_id == {row['user_id']}""")
     .groupby(['school_year', 'term'])
     .nunique()
     .index
    ) 
    if duration >= 2 and row['unweighted_gpa'] >= 3:
        qualified_students.append(row['user_id'])
    

In [ ]:
honors_students = records['gpas'][records['gpas'].user_id.isin(qualified_students)].sort_values('unweighted_gpa')

In [ ]:
students = sky.getUsers()
students = (students[['id', 'first_name', 'last_name', 'student_info.grade_level_abbreviation']]
            .rename(columns={
                'id':'user_id',
                'student_info.grade_level_abbreviation':'grade_level'
            })
           )

In [ ]:
honors_students = (
    students
    .merge(
        honors_students.drop('weighted_gpa', axis=1)
    )
)

In [ ]:
honors_students.sort_values('unweighted_gpa')

In [ ]:
# createSpreadsheet(honors_students.drop('user_id', axis=1).sort_values('unweighted_gpa'), 
#                   'NJHS Report',
#                   styleClass=HysonFireStyle
#                  )
# id = 1JId_rzdZcHZB3MIepnabrXxdo8Mp7nAYz8xKnu6bLbQ

In [ ]:
sp = sky.getAdvancedList(74475)
sp.drop(['firstname', 'lastname'], axis=1, inplace=True)

In [ ]:
data = (
    honors_students
    .merge(
        sp.astype({'user_id':'int64'})
    )
    .drop('user_id', axis=1)
)

In [ ]:
updateSpreadsheet(data.fillna(''), sheet_id='1JId_rzdZcHZB3MIepnabrXxdo8Mp7nAYz8xKnu6bLbQ')